## Log likelihood benchmark

tested: 
0. numpy + scipy's pdf
1. numpy
2. cython
4. numexpr
3. theano
4. parakeet

computing log-likelihood for normal distribution 

__Notes__
1. Not optimizing computations here (but in theory theano and parakeet may remove unnecessary computations)
2. This test includes exp, log, division and summation of array
2. Everything is running on CPU in one thread, this limitation is for 'fairness' of tests


In [1]:
import numpy
import scipy
from scipy.stats import norm
import theano

### Scipy implementation

In [2]:
def llh_scipy(data, mean, sigma):
    lh = norm(mean, sigma).pdf(data)
    return numpy.log(lh).sum()

### Numpy implementation

In [3]:
def llh_numpy(data, mean, sigma):
    s = (data - mean) ** 2 / (2 * (sigma ** 2))
    pdfs = numpy.exp(- s)
    pdfs /= numpy.sqrt(2 * numpy.pi) * sigma
    return numpy.log(pdfs).sum()

### Cython implementation

In [4]:
%load_ext Cython

In [5]:
%%cython
cdef extern from "math.h":
    double sqrt(double m)
    double exp(double m)
    double log(double m)

import cython
import numpy as np

cimport numpy as np
from numpy cimport ndarray
pi = np.pi

@cython.boundscheck(False)
@cython.wraparound(False)
def llh_cython(ndarray[np.float64_t, ndim=1] data, double mean, double sigma):
    cdef int l = len(data)
    cdef double llh = 0
    cdef double s = 0
    for i in xrange(l):
        s = (data[i] - mean) ** 2 / (2 * (sigma ** 2))
        llh += log(exp(-s) / (sqrt(2 * pi) * sigma))
    return llh

### Numexpr implementation

In [6]:
import numexpr
numexpr.set_num_threads(1)

def llh_numexpr(data, mean, sigma):
    expression = 'sum(log(exp(- (data-mean) **2  / (2 * sigma ** 2)) / (sqrt(2 * pi) * sigma)))'
    return numexpr.evaluate(expression, local_dict=dict(data=data, mean=mean, sigma=sigma, pi=numpy.pi))

### Parakeet implementation

In [7]:
import parakeet
parakeet.config.backend = 'c'

@parakeet.jit 
def llh_parakeet(data, mean, sigma):
    s = (data - mean) ** 2 / (2 * (sigma ** 2))
    pdfs = numpy.exp(- s)
    pdfs /= numpy.sqrt(2 * numpy.pi) * sigma
    return numpy.log(pdfs).sum()

### Theano implementation

In [8]:
import theano
import theano.tensor as T

print theano.config.device

cpu


In [9]:
theano.config.openmp

False

In [10]:
def llh_theano(data, mean, sigma):
    s = (data - mean) ** 2 / (2 * (sigma ** 2))
    pdfs = T.exp(- s)
    pdfs /= T.sqrt(2 * numpy.pi) * sigma
    return T.log(pdfs).sum()

mean, sigma = T.scalars('m', 's')
d = T.vector('data')

llh_theano = theano.function([d, mean, sigma], llh_theano(d, mean, sigma))

## C++ implementation for comparison of speed
we are neither passing, nor returning anything in c++. Just doing same operations in C++ for some array to compare speed.

Mind the overhead for creating new process - it is essential for small sizes.

In [11]:
%%writefile test_speed.cpp
#include <iostream>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

// using namespace std;


int main(int argc, char** argv) {
    if (argc < 4){
        std::cout << "run with n_samples, mean, sigma!";
        return 1;
    }
    int size = atoi(argv[1]);
    double mean = atof(argv[2]);
    double sigma = atof(argv[3]);
    
    double * data = new double[size];
    double factor = 1. / size;
    for (int i=0; i<size; ++i){
        data[i] = i * factor;
    }
    double result = 0.;
    double s = 0.;
    double x = 0.;
    
    for (int i=0; i<size; ++i){
        x = (data[i] - mean);
        s =  x * x / (2 * (sigma * sigma));
        result += log(exp(-s) / (sqrt(2 * M_PI) * sigma));
    }
    
    std::cout << std::endl << result << std::endl;
    return 0;
}

Overwriting test_speed.cpp


In [12]:
!g++ test_speed.cpp -o test_speed -O3

In [13]:
def llh_cpp(data, mean, sigma):
    size = len(data)
    out = !./test_speed {len(data)} {mean} {sigma}

### Data generation, checking that all functions output the same value

In [14]:
from collections import OrderedDict
functions = OrderedDict()
functions['scipy'] = llh_scipy
functions['numpy'] = llh_numpy
functions['cython'] = llh_cython
functions['numexpr'] = llh_numexpr
functions['parakeet'] = llh_parakeet
functions['theano'] = llh_theano
functions['c++'] = llh_cpp

In [15]:
data = numpy.random.normal(size=1000000).astype('float64')
for name, function in functions.items():
    print name, function(data, 0.1, 1.1)

scipy -1431620.79845
numpy -1431620.79845
cython -1431620.79845
numexpr -1431620.79845
parakeet -1431620.79845
theano -1431620.79845
c++ None


In [16]:
import timeit 
sizes = [10 ** 5, 10 ** 6, 10 ** 7, 10 ** 8]
import pandas
scores = pandas.DataFrame(data=0, columns=functions.keys(), index=sizes)
for size in sizes:
    for name, function in functions.items():
        data = numpy.random.normal(size=size).astype('float64')
        result = %timeit -o function(data, 0.1, 1.1)
        scores.loc[size, name] = result.best

100 loops, best of 3: 10.9 ms per loop
100 loops, best of 3: 5.97 ms per loop
100 loops, best of 3: 10.6 ms per loop
100 loops, best of 3: 7.01 ms per loop
100 loops, best of 3: 6.32 ms per loop
100 loops, best of 3: 7.7 ms per loop
100 loops, best of 3: 17 ms per loop
10 loops, best of 3: 131 ms per loop
10 loops, best of 3: 62.1 ms per loop
10 loops, best of 3: 109 ms per loop
10 loops, best of 3: 72.6 ms per loop
10 loops, best of 3: 64.5 ms per loop
10 loops, best of 3: 80 ms per loop
10 loops, best of 3: 79.8 ms per loop
1 loops, best of 3: 1.8 s per loop
1 loops, best of 3: 840 ms per loop
1 loops, best of 3: 1.07 s per loop
1 loops, best of 3: 700 ms per loop
1 loops, best of 3: 628 ms per loop
1 loops, best of 3: 822 ms per loop
1 loops, best of 3: 697 ms per loop
1 loops, best of 3: 18.9 s per loop
1 loops, best of 3: 8.54 s per loop
1 loops, best of 3: 10.9 s per loop
1 loops, best of 3: 7.16 s per loop
1 loops, best of 3: 6.25 s per loop
1 loops, best of 3: 8.23 s per loop
1

## Results (time in seconds, less is better) 

In [17]:
scores

,scipy,numpy,cython,numexpr,parakeet,theano,c++
100000,0.010891,0.005968,0.010637,0.007012,0.006323,0.007702,0.017023
1000000,0.130639,0.062146,0.108749,0.072568,0.064520,0.079974,0.079771
10000000,1.795048,0.840145,1.067230,0.700086,0.627737,0.821548,0.696519
100000000,18.917289,8.539139,10.910751,7.157359,6.250242,8.229951,6.924702


## Comparison to numpy time (less is better)

In [18]:
normalized_scores = scores.copy()
for column in normalized_scores.columns:
    normalized_scores[column] /= scores['numpy']    

In [19]:
normalized_scores

,scipy,numpy,cython,numexpr,parakeet,theano,c++
100000,1.824860,1,1.782218,1.174857,1.059506,1.290531,2.852221
1000000,2.102137,1,1.749897,1.167704,1.038194,1.286867,1.283616
10000000,2.136593,1,1.270292,0.833292,0.747177,0.977864,0.829046
100000000,2.215363,1,1.277734,0.838183,0.731952,0.963792,0.810937


## Conclusion

Many libraries claim that they can speed up number crunching in python.
Results of this test are floating (+- 0.1), but what we can see
1. numpy turned out to be fastest at moderate sizes of arrays
2. numpy implementation at least not more complex than others
3. parakeet was the only to get sensible speed up

# Technical info

In [20]:
import multiprocessing
print multiprocessing.cpu_count(), 'xeon cores'

16 xeon cores


In [21]:
numpy.__version__

'1.9.2'

In [22]:
scipy.__version__

'0.14.0'

In [23]:
import cython
cython.__version__

'0.21.1'

In [24]:
numexpr.__version__

'2.4'

In [25]:
parakeet.__version__

'0.23.2'

In [26]:
theano.__version__

'0.7.0'

In [27]:
!g++ -v

Using built-in specs.
COLLECT_GCC=g++
COLLECT_LTO_WRAPPER=/usr/lib/gcc/x86_64-linux-gnu/4.6/lto-wrapper
Target: x86_64-linux-gnu
Configured with: ../src/configure -v --with-pkgversion='Ubuntu/Linaro 4.6.3-1ubuntu5' --with-bugurl=file:///usr/share/doc/gcc-4.6/README.Bugs --enable-languages=c,c++,fortran,objc,obj-c++ --prefix=/usr --program-suffix=-4.6 --enable-shared --enable-linker-build-id --with-system-zlib --libexecdir=/usr/lib --without-included-gettext --enable-threads=posix --with-gxx-include-dir=/usr/include/c++/4.6 --libdir=/usr/lib --enable-nls --with-sysroot=/ --enable-clocale=gnu --enable-libstdcxx-debug --enable-libstdcxx-time=yes --enable-gnu-unique-object --enable-plugin --enable-objc-gc --disable-werror --with-arch-32=i686 --with-tune=generic --enable-checking=release --build=x86_64-linux-gnu --host=x86_64-linux-gnu --target=x86_64-linux-gnu
Thread model: posix
gcc version 4.6.3 (Ubuntu/Linaro 4.6.3-1ubuntu5) 
